# Introduction
This is a test for doing some basic ETL on data.

## The Goal
The goal of this task is to take a pair excel files, load them into SQL, and then transform the dataset into a separate schema (called FHIR).

### Input data
The input data is an pair of excel files with patient information in it. Each file represents an extract of a data system, each made a month apart. Each file includes patients discharged during the last two months, so the data will have intersecting data points, but the union of the two is really what you want.  In the boilerplate code below, you can see the file paths and open them up in excel to browse it if you'd like. Feel free to ask questions about the file formatting.

# Exercise

## Load into SQL
First, begin by loading the data into a SQL database. We have Postgres running for you already, so use that. 
Username: carta
Password: password

The data should be a union of the data in both extract files, with an update time corresponding to the newest update date available for each row.

## Transform into FHIR

After the data is loaded into SQL, you'll produce some FHIR resources from the data. In particular, there are two resources you'll make:

1) Patient
2) Encounter

Both of those are defined on the FHIR website, here: http://fhir.org. We will do a validation on the data once it's done!


In [1]:
import pandas as pd
import numpy as np
from carta_interview import Datasets, get_data_file

In [2]:
patient_extract1 = get_data_file(Datasets.PATIENT_EXTRACT1)
patient_extract2 = get_data_file(Datasets.PATIENT_EXTRACT2)

In [4]:
import psycopg2
import xlrd
import pandas as pd
import numpy as np
from carta_interview import Datasets, get_data_file
import datetime
import pprint

patient_extract1 = get_data_file(Datasets.PATIENT_EXTRACT1)
patient_extract2 = get_data_file(Datasets.PATIENT_EXTRACT2)

# To open Workbook 
wb1 = xlrd.open_workbook(patient_extract1) 
sheet1 = wb1.sheet_by_index(0) 

wb2 = xlrd.open_workbook(patient_extract2) 
sheet2 = wb2.sheet_by_index(0) 

con = psycopg2.connect(host='localhost', database='postgres',
                       user='carta', password='password')

cur = con.cursor()

cur.execute('CREATE TABLE data ( \
	id serial NOT NULL, \
	mrn varchar(4) NOT NULL, \
	encounter_id int NOT NULL, \
	first_name varchar(30) NOT NULL, \
	last_name varchar(30) NOT NULL, \
	birthdate date NOT NULL, \
	admission_dt timestamp NOT NULL, \
	discharge_dt timestamp NOT NULL,\
	update_dt timestamp NOT NULL, \
	CONSTRAINT data_pk PRIMARY KEY (id) \
);')


for index in range(1, sheet1.nrows):
    cur.execute(f"INSERT INTO data(\
                    mrn, encounter_id, first_name, last_name, birthdate, admission_dt, discharge_dt, \
                    update_dt) \
                VALUES({sheet1.cell_value(index, 0)}, \
                        {sheet1.cell_value(index, 1)}, \
                        '{sheet1.cell_value(index, 2)}', \
                        '{sheet1.cell_value(index, 3)}', \
                        '{sheet1.cell_value(index, 4)}', \
                        '{sheet1.cell_value(index, 5)}', \
                        '{sheet1.cell_value(index, 6)}', \
                        '{sheet1.cell_value(index, 7)}')")
    
for index in range(1, sheet2.nrows):
    # check if there is a data with the same encounter_id
    cur.execute(f'SELECT update_dt FROM data WHERE encounter_id = {sheet2.cell_value(index, 1)}')
    exists = cur.fetchone()
    
    # format the string to date
    admission_date = datetime.datetime.strptime(sheet2.cell_value(index, 5), "%m/%d/%Y %I:%M %p")
    discharge_date = datetime.datetime.strptime(sheet2.cell_value(index, 6), "%m/%d/%Y %I:%M %p")
    update_date = datetime.datetime.strptime(sheet2.cell_value(index, 7), "%m/%d/%Y %I:%M %p")
    
    if not exists:    
        cur.execute(f"INSERT INTO data(\
                        mrn, encounter_id, first_name, last_name, birthdate, admission_dt, discharge_dt, \
                        update_dt) \
                    VALUES({sheet2.cell_value(index, 0)}, \
                            {sheet2.cell_value(index, 1)}, \
                            '{sheet2.cell_value(index, 2)}', \
                            '{sheet2.cell_value(index, 3)}', \
                            '{sheet2.cell_value(index, 4)}', \
                            '{admission_date}', \
                            '{discharge_date}', \
                            '{update_date}')")
    elif update_date > exists[0]:
        cur.execute(f"UPDATE data \
                    SET update_dt = '{update_date}' \
                    WHERE encounter_id = {sheet2.cell_value(index, 1)}")
        
cur.execute(f'''SELECT mrn,
                        encounter_id,
                        first_name,
                        last_name,
                        birthdate,
                        admission_dt,
                        discharge_dt,
                        update_dt
                FROM data''')
records = cur.fetchall()

patient_resources = []
encounter_resources = []

#mount the resources
for r in records:
    patient_resources.append({
        "resourceType" : "Patient",
        "identifier" : [r[0]],
        "name" : [ ' '.join([r[2],r[3]]) ], 
        "gender" : "unknown", 
        "birthDate" : r[4],
    })
    
    encounter = ({
      "resourceType" : "Encounter",
      "identifier" : [r[1]],
      "status" : "finished",
      "class" : "AMB"
    })
    encounter["statusHistory"] = []
    encounter["statusHistory"].append({
        "status" : "arrived", 
        "period" : r[5] 
      })
    encounter["statusHistory"].append({
        "status" : "onleave", 
        "period" : r[6] 
      })

    encounter_resources.append(encounter)
    
    
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(patient_resources)
pp.pprint(encounter_resources)

[   {   'birthDate': datetime.date(1999, 1, 2),
        'gender': 'unknown',
        'identifier': ['1'],
        'name': ['John Doe'],
        'resourceType': 'Patient'},
    {   'birthDate': datetime.date(1988, 1, 2),
        'gender': 'unknown',
        'identifier': ['20'],
        'name': ['Jerry Jones'],
        'resourceType': 'Patient'},
    {   'birthDate': datetime.date(2002, 4, 19),
        'gender': 'unknown',
        'identifier': ['2'],
        'name': ['Joanne Lee'],
        'resourceType': 'Patient'},
    {   'birthDate': datetime.date(2001, 1, 2),
        'gender': 'unknown',
        'identifier': ['3'],
        'name': ['Annabelle Jones'],
        'resourceType': 'Patient'}]
[   {   'class': 'AMB',
        'identifier': [1234],
        'resourceType': 'Encounter',
        'status': 'finished',
        'statusHistory': [   {   'period': datetime.datetime(2002, 4, 12, 17, 0),
                                 'status': 'arrived'},
                             {   'period